## 前言
1. 本项目从Iron_Simulation中分离而来。旨在为统一的投机采样建模Roofline模型。  
2. 首先，我固定一个模型：llama-3.1-8B。
备注：其实不同的算法还是使用的不同版本，比如eagle-3和eagle-2用的是llama-3.1-8b-instruct模型，LongSpec用的是一个262k的模型，而MagicDec用的是原始的模型。
但是这只会导致算法上的接受率有点差异，而不会影响我硬件的建模。
3. 然后，我仿照Iron_Simulation项目中子文件夹LongMagic和子文件夹Roofline_model的思路，创建不同的大、小模型计算文件。放在./modeling这个子文件夹中。结构如下：
```
├── modeling_llama3.py # 里面的函数都要支持kv cache和batch size > 1的情况
│   ├── llama3_weight_load_size() 
│   ├── llama3_act_load_size()
│   ├── llama3_act_st_size()
│   ├── llama3_mm_comp()
│   └── llama3_cycles_comp()
├── modeling_eagle2.py # 针对llama3的eagle2模型，主要是GQA的结构
│   ├── eagle2_weight_load_size()
│   ├── eagle2_act_load_size()
│   ├── eagle2_act_st_size()
│   ├── eagle2_mm_comp()
│   ├── eagle2_cycles_comp()
│   └── eagle2_draft_cycles_comp()
├── modeling_eagle3.py # 针对llama3的eagle3模型
│   ├── eagle3_weight_load_size()
│   ├── eagle3_act_load_size()
│   ├── eagle3_act_st_size()
│   ├── eagle3_mm_comp()
│   ├── eagle3_cycles_comp()
│   └── eagle3_draft_cycles_comp
├── modeling_longspec.py # 针对llama3的longspec小模型
│   ├── longspec_weight_load_size()
│   ├── longspec_act_load_size()
│   ├── longspec_act_st_size()
│   ├── longspec_mm_comp()
│   ├── longspec_cycles_comp()
│   └── longspec_draft_cycles_comp()
├── parse_args.py # 提供基本的超参数
│   └── parse_args()
├── parse_args.py #存储算法上跑出的aat数据。
# 其他算法的小模型支持，比如SpS！
# MagicDec不需要单独的模型
```
4. 接下来，在子文件夹./roofline中，我需要创建一些用来运行的文件。结构如下：
```
├── run_baseline.py     # 获得不使用投机采样算法跑大模型的性能
├── run_eagle2.py       # 获得eagle2算法在不同total-token配置下的性能
├── run_eagle3.py       # 获得eagle3算法在不同total-token配置下的性能
├── run_longspec.py
├── run_magicdec.py
├── run_sps.py          # 找一个现成的小模型，比如llama-3.2-1B.
├── draw.py # 用来绘制roofline模型以及组合模型的曲线
│   ├── draw_roofline() 
│   └── draw_combined_roofline()
│   └── 同时绘制多个模型的曲线
```

To do:
1. 好好研究LongSpec，还要围绕它多做点实验。比如它在batch size较大时效果如何？MagicSpec在这里面是怎么实现的？有SpS的实现吗？它如何控制verify的token数？
回答：目前有spec的实现，也就是链式投机采样。我现在先跑一个链式的结果。
2. EAGLE3在batch size大于1时是怎么实现的？计算量乘以batch size就行吗？
回答：它代码不能直接跑，我暂时不想去修改代码，所以计算量就直接乘以batch size吧。
3. 模型的统一？
4. 测试的数据集的统一？看起来用PG-19 dataset是一个不错的选择。
5. 我先把目前的4个算法的图都绘制完成。EAGLE-2和EAGLE-3针对短文本，LongSpec和MagicDec针对长文本。